In [18]:
import pandas as pd
import numpy as np
import joblib  # For saving the model
from sqlalchemy import create_engine  # For connecting to the database
from sklearn.ensemble import RandomForestRegressor  # Random Forest model
from sklearn.model_selection import train_test_split  # For splitting data
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # For model evaluation
import matplotlib.pyplot as plt  # For plotting
from sklearn.decomposition import PCA  # For dimensionality reduction
from dotenv import load_dotenv
import os
from scipy.sparse import csr_matrix  # For sparse matrix representation

# Load environment variables from .env file
load_dotenv()

# Set up the database connection using environment variables
db_url = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(db_url)

# SQL Query to fetch the required data
query = """
SELECT
    s.salesid,
    s.invoiceno,
    s.customerid,
    s.stockcode,
    s.quantity,
    s.unitprice,
    s.totalprice,
    t.day,
    t.month,
    t.year,
    t.hour,
    t.minute,
    c.country
FROM
    sales s
JOIN
    time t ON s.timeid = t.timeid
JOIN
    customer c ON s.customerid = c.customerid
"""

# Load the data from the database
df = pd.read_sql(query, engine)

# Inspect the first few rows of the data
print("\nFirst few rows of the data:")
print(df.head())

# Create a new column for the date
df['Date'] = pd.to_datetime(df[['year', 'month', 'day']])

# Feature Engineering for the model (weekly aggregation)
df['Week'] = df['Date'].dt.isocalendar().week
df['Year'] = df['Date'].dt.year

# Aggregating data by year and week
df_weekly = df.groupby(['Year', 'Week']).agg(
    total_quantity_ordered=('quantity', 'sum'),
    total_sales_value=('totalprice', 'sum'),
    avg_unitprice=('unitprice', 'mean')
).reset_index()

# Feature Engineering for weekly data
df_weekly['DateNumeric'] = (df_weekly['Year'].astype(str) + '-' + df_weekly['Week'].astype(str)).astype('category').cat.codes
df_weekly['is_weekend'] = df_weekly['Week'].isin([5, 6]).astype(int)  # Assuming weekends are 5 and 6

# Lag feature for previous week's total sales
df_weekly['sales_lag_1'] = df_weekly['total_sales_value'].shift(1)
df_weekly['sales_lag_2'] = df_weekly['total_sales_value'].shift(2)  # Add 2nd lag feature
df_weekly.dropna(inplace=True)  # Drop NaN rows after lagging

# One-hot encoding for 'is_weekend'
df_weekly = pd.get_dummies(df_weekly, columns=['is_weekend'], drop_first=True)

# Feature engineering for time of year
df_weekly['is_q1'] = (df_weekly['DateNumeric'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%U').month).isin([1, 2, 3])).astype(int)
df_weekly['is_q2'] = (df_weekly['DateNumeric'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%U').month).isin([4, 5, 6])).astype(int)
df_weekly['is_q3'] = (df_weekly['DateNumeric'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%U').month).isin([7, 8, 9])).astype(int)
df_weekly['is_q4'] = (df_weekly['DateNumeric'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%U').month).isin([10, 11, 12])).astype(int)

# Time-related features
df_weekly['is_holiday_season'] = df_weekly['DateNumeric'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%U').month).isin([11, 12]).astype(int)  # Holiday season
df_weekly['day_of_week'] = df_weekly['DateNumeric'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%U').dayofweek)  # 0 = Monday, 6 = Sunday

# Adding rolling statistics features
df_weekly['rolling_avg_sales'] = df_weekly['total_sales_value'].rolling(window=4).mean()  # 4-week rolling average
df_weekly['rolling_std_sales'] = df_weekly['total_sales_value'].rolling(window=4).std()  # 4-week rolling std deviation

# Adding customer-related features (aggregates by customer)
customer_sales = df.groupby('customerid').agg(
    total_customer_sales=('totalprice', 'sum'),
    avg_customer_spend=('totalprice', 'mean'),
    total_customer_orders=('salesid', 'count')
).reset_index()

# Merging customer sales data with weekly data
df_weekly = df_weekly.merge(customer_sales, on='customerid', how='left')

# Define features and target
features = ['DateNumeric', 'avg_unitprice', 'sales_lag_1', 'sales_lag_2', 'is_q1', 'is_q2', 'is_q3', 'is_q4', 
            'is_weekend_1', 'is_holiday_season', 'day_of_week', 'rolling_avg_sales', 'rolling_std_sales', 
            'total_customer_sales', 'avg_customer_spend', 'total_customer_orders']
X = df_weekly[features]
y = df_weekly['total_sales_value']  # Predicting total sales value per week

# Get the number of features in X
n_features = X.shape[1]

# Set n_components to the minimum of 50 or the number of features in X
pca = PCA(n_components=min(50, n_features))
X_pca = pca.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model with fewer estimators
model = RandomForestRegressor(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the performance metrics
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

# Save the model for future use
joblib.dump(model, 'weekly_sales_forecasting_rf_model.pkl')
print("Model saved as 'weekly_sales_forecasting_rf_model.pkl'")

# Visualize the results with a chart for Actual vs Predicted Sales (weekly)
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test.values, label='Actual Sales Value', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted Sales Value', color='red', linestyle='dashed')
plt.xlabel('Sample Index')
plt.ylabel('Total Sales Value')
plt.title('Actual vs Predicted Total Sales Value (Weekly)')
plt.legend()
plt.grid(True)
plt.show()

# Visualize a bar chart showing actual vs predicted sales for the first 20 test samples
plt.figure(figsize=(10, 6))

# Adjust sample indices to match the size of the y_test_sample
sample_indices = np.arange(min(20, len(y_test)))  # Adjust sample indices based on y_test size

# Select the first 20 samples for actual and predicted values (if available)
y_test_sample = y_test.head(len(sample_indices)).values
y_pred_sample = y_pred[:len(sample_indices)]

# Plot the bar chart
plt.bar(sample_indices, y_test_sample, label='Actual Sales Value', color='blue', alpha=0.6)
plt.bar(sample_indices, y_pred_sample, label='Predicted Sales Value', color='red', alpha=0.6)

plt.xlabel('Sample Index')
plt.ylabel('Total Sales Value')
plt.title(f'Bar Chart: Actual vs Predicted Sales Value (First {len(sample_indices)} Samples)')
plt.legend()
plt.show()



First few rows of the data:
   salesid invoiceno  customerid stockcode  quantity  unitprice  totalprice  \
0        1    536365       17850    85123A         6       2.55       15.30   
1        2    536365       17850     71053         6       3.39       20.34   
2        3    536365       17850    84406B         8       2.75       22.00   
3        4    536365       17850    84029G         6       3.39       20.34   
4        5    536365       17850    84029E         6       3.39       20.34   

   day  month  year  hour  minute         country  
0    1     12  2010     8      26  United Kingdom  
1    1     12  2010     8      26  United Kingdom  
2    1     12  2010     8      26  United Kingdom  
3    1     12  2010     8      26  United Kingdom  
4    1     12  2010     8      26  United Kingdom  


ValueError: Cannot use '%W' or '%U' without day and year